In [ ]:
%cd /home/ubuntu/Qwen-Image-Edit-Angles

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
from qwenimage.experiment import ExperimentConfig
from qwenimage.experiments.experiments_qwen import ExperimentRegistry


# experiment_names = ExperimentRegistry.keys()
# experiment_names = [
#     "qwen_base",
#     # "qwen_fa3",
#     # "qwen_aot",
#     # "qwen_fa3_aot",
#     # "qwen_fa3_aot_int8",
#     # "qwen_fa3_aot_fp8",
#     # "qwen_fuse",
#     "qwen_base_3step",
#     "qwen_base_2step",
#     "qwen_lightning_lora",
#     "qwen_lightning_lora_3step",
#     "qwen_lightning_lora_2step",
# ]

experiment_names = [
    # "qwen_fa3",
    # "qwen_aot",
    # "qwen_fa3_aot",
    "qwen_fa3_aot_int8",
    "qwen_fa3_aot_int8_fuse",
    "qwen_lightning_fa3_aot_int8_fuse",
    # "qwen_fa3_fuse",
    "qwen_fa3_aot_fp8",
    "qwen_fa3_aot_fp8_fuse",
    "qwen_lightning_fa3_aot_fp8_fuse",
    "qwen_lightning_fa3_aot_int8_fuse_2step",
    "qwen_base",
]

report_dir = ExperimentConfig().report_dir

experiment_outputs = {}
num_outputs = {}
for name in experiment_names:
    output_dir = report_dir / f"{name}_outputs"
    all_output_paths = sorted(list(output_dir.glob("*.jpg")))
    experiment_outputs[name] = all_output_paths
    num_outputs[name] = len(all_output_paths)




In [ ]:
experiment_outputs["qwen_base"][0]

In [ ]:
import math
from PIL import Image
import numpy as np
from pathlib import Path

comparison_dir = report_dir / "comparison_grid_2"
comparison_dir.mkdir(exist_ok=True, parents=True)


comparable_outputs = min(num_outputs.values())
num_experiments = len(experiment_names)

# For each output index, create a comparison grid
for idx in range(comparable_outputs):
    cols = int(math.ceil(math.sqrt(num_experiments)))
    rows = int(math.ceil(num_experiments / cols ))
    
    fig, axes = plt.subplots(rows, cols, figsize=(6 * cols, 6 * rows))
    
    # Flatten axes array for easier indexing
    if num_experiments == 1:
        axes = np.array([axes])
    else:
        axes = axes.flatten() if isinstance(axes, np.ndarray) else np.array([axes])
    
    # Plot each experiment's output for this index
    for exp_idx, exp_name in enumerate(experiment_names):
        ax = axes[exp_idx]
        
        # Check if this experiment has an output at this index
        if idx < num_outputs[exp_name]:
            img_path = experiment_outputs[exp_name][idx]
            img = Image.open(img_path)
            ax.imshow(img)
            ax.set_title(exp_name, fontsize=14, fontweight='bold')
        else:
            # No output for this experiment at this index
            ax.text(0.5, 0.5, 'N/A', ha='center', va='center', fontsize=20)
            ax.set_title(exp_name, fontsize=14, fontweight='bold', color='gray')
        
        ax.axis('off')
    
    # Hide any unused subplots
    for exp_idx in range(num_experiments, len(axes)):
        axes[exp_idx].axis('off')
    
    # Add a main title for the figure
    fig.suptitle(f'Output Comparison - Index {idx:03d}', fontsize=18, fontweight='bold', y=0.98)
    
    plt.tight_layout()
    
    # Save the figure
    output_path = comparison_dir / f"comparison_{idx:03d}.jpg"
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close(fig)
    
    print(f"Saved comparison grid {idx + 1}/{comparable_outputs} to {output_path}")

print(f"\nAll comparison grids saved to {comparison_dir}")
